## 1783. Grand Slam Titles
### Table: Players

| Column Name | Type    |
|-------------|---------|
| player_id   | int     |
| player_name | varchar |

player_id is the primary key for this table.  
Each row in this table contains the name and the ID of a tennis player.

---

### Table: Championships

| Column Name | Type    |
|-------------|---------|
| year        | int     |
| Wimbledon   | int     |
| Fr_open     | int     |
| US_open     | int     |
| Au_open     | int     |

year is the primary key for this table.  
Each row of this table containts the IDs of the players who won one each tennis tournament of the grand slam.

---

### Problem Statement

Write an SQL query to report the number of grand slam tournaments won by each player. Do not include the players who did not win any tournament.

Return the result table in any order.

---

### Sample Input

#### Players table:

| player_id | player_name |
|-----------|-------------|
| 1         | Nadal       |
| 2         | Federer     |
| 3         | Novak       |

#### Championships table:

| year | Wimbledon | Fr_open | US_open | Au_open |
|------|-----------|---------|---------|---------|
| 2018 | 1         | 1       | 1       | 1       |
| 2019 | 1         | 1       | 2       | 2       |
| 2020 | 2         | 1       | 2       | 2       |

---

### Output

| player_id | player_name | grand_slams_count |
|-----------|-------------|-------------------|
| 2         | Federer     | 5                 |
| 1         | Nadal       | 7                 |

---

### Explanation

Player 1 (Nadal) won 7 titles: Wimbledon (2018, 2019), Fr_open (2018, 2019, 2020), US_open (2018), and Au_open (2018).  
Player 2 (Federer) won 5 titles: Wimbledon (2020), US_open (2019, 2020), and Au_open (2019, 2020).  
Player 3 (Novak) did not win anything, we did not include them in the result table.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

spark = SparkSession.builder.getOrCreate()

# Define schema
players_schema = StructType([
    StructField("player_id", IntegerType(), True),
    StructField("player_name", StringType(), True)
])

championships_schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("Wimbledon", IntegerType(), True),
    StructField("Fr_open", IntegerType(), True),
    StructField("US_open", IntegerType(), True),
    StructField("Au_open", IntegerType(), True)
])

# Sample data
players_data = [
    (1, "Nadal"),
    (2, "Federer"),
    (3, "Novak")
]

championships_data = [
    (2018, 1, 1, 1, 1),
    (2019, 1, 1, 2, 2),
    (2020, 2, 1, 2, 2)
]

# Create DataFrames
players_df = spark.createDataFrame(players_data, schema=players_schema)
championships_df = spark.createDataFrame(championships_data, schema=championships_schema)

# Register temp views
players_df.createOrReplaceTempView("Players")
championships_df.createOrReplaceTempView("Championships")


In [0]:

# SQL logic
query = """
SELECT 
    p.player_id,
    p.player_name,
    COUNT(*) AS grand_slams_count
FROM (
    SELECT Wimbledon AS player_id FROM Championships
    UNION ALL
    SELECT Fr_open FROM Championships
    UNION ALL
    SELECT US_open FROM Championships
    UNION ALL
    SELECT Au_open FROM Championships
) AS wins
JOIN Players p ON wins.player_id = p.player_id
GROUP BY p.player_id, p.player_name
"""

result_df = spark.sql(query)
display(result_df)